In [1]:
# Set notebook to use only one GPU
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_VISIBLE_DEVICES=1


# Predicting

Generate and save Y_pred for both methods


In [7]:
from file_functions import *
from models_bachelors import *
import plotly.express as px
import plotly.figure_factory as ff
import pandas as pd
from keras_uncertainty.models import StochasticClassifier

dataset = load('all_subject_runs')
loaded_inputs = dataset['inputs']
loaded_targets = dataset['targets']
'''
Loads a dictionary with 2 keys: 'inputs', 'targets'. 
Both keys have ndarray containing inputs and targets
of 9 subjects separated by subject.
'''
lockbox = load('lockbox')

# To-do:

This code was originally intended to save both methods predictions. But MCDropout had to undergo training again because turns out it was actually MCDropConnect being trained 💀💀💀. So now the code only works when methods has a single element.

In [3]:
predictions = {'mcdropout': 
               {'preds': {'test':[], 'lockbox':[]}, 
                'labels':{'test':[], 'lockbox':[]}},
                'mcdropconnect': 
                {'preds': {'test':[], 'lockbox':[]}, 
                'labels':{'test':[], 'lockbox':[]}}
              }

for method, values in predictions.items():
    wts_directory = f'{method}/weights'
    # For each subject, get Y_preds for the test subject, and also get the 
    # Y_preds for the lockboxed data. Then save
    # Y preds for each subject would be shape (456, 4)
    for subject_id in range(0, 9):
        X_test = loaded_inputs[subject_id]
        Y_true = loaded_targets[subject_id]
        wts_path = checkpoint_path = f'{wts_directory}/test_subject_{subject_id}.ckpt'
        model = create_model(method=method, drop_rates=0.2)
        model.load_weights(wts_path)
        # Get Y_preds for test subject
        if method != 'standard':
            model = StochasticClassifier(model)
            Y_preds = model(X_test, num_samples=50)
            # Get lockboxed Y_preds for test subject
            lockbox_Y_preds = model(lockbox['inputs'][subject_id], num_samples=50)
        else:
            Y_preds = model.predict(X_test)
            # Get lockboxed Y_preds for test subject
            lockbox_Y_preds = model(lockbox['inputs'][subject_id])

        '''
          Must append Y_true for both cases too!
        '''
        lockbox_Y_true = lockbox['targets'][subject_id]
        values['preds']['test'].append(Y_preds)
        values['preds']['lockbox'].append(lockbox_Y_preds)
        values['labels']['test'].append(Y_true)
        values['labels']['lockbox'].append(lockbox_Y_true)

In [8]:
for method, values in predictions.items():
    values['preds']['test'] = np.array(values['preds']['test'])
    values['preds']['lockbox'] = np.array(values['preds']['lockbox'])
    values['labels']['test'] = np.array(values['labels']['test'])
    values['labels']['lockbox'] = np.array(values['labels']['lockbox'])

dict2hdf5('mcdropout/predictions_mcdropout.h5', predictions['mcdropout'])
dict2hdf5('mcdropconnect/predictions_mcdropconnect.h5', predictions['mcdropconnect'])


mcdropout predictions on test: (9, 576, 4)
mcdropout predictions on lockbox: (9, 456, 4)
mcdropout labels on test: (9, 576, 4)
mcdropout labels on lockbox: (9, 456, 4)
mcdropconnect predictions on test: (9, 576, 4)
mcdropconnect predictions on lockbox: (9, 456, 4)
mcdropconnect labels on test: (9, 576, 4)
mcdropconnect labels on lockbox: (9, 456, 4)
